# Vodní díla v Jihomoravském kraji

In [ ]:
###
import geopandas
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx

Vytvoříme databázový dotaz na všechu vodu, která je umělá nádrž nebo jezero.  S tvorbou dotazu nám pomůže dokumentace OpenStreetMaps, která pro ČR obsahuje i Registr územní identifikace, adres a nemovitostí [RUIAN](https://wiki.openstreetmap.org/wiki/Cs:R%C3%9AIAN#Parcely)

`(water=reservoir or water=lake) in "Jihomoravský kraj"`

Tento dotaz vložíme do průvodce na http://overpass-turbo.eu/ a vytvoří se dotaz v jazyce **Overpass**.

```overpass
/*
This has been generated by the overpass-turbo wizard.
The original search was:
“(water=reservoir or water=lake) in "Jihomoravský kraj"”
*/
[out:json][timeout:25];
// fetch area “Jihomoravský kraj” to search in
{{geocodeArea:Jihomoravský kraj}}->.searchArea;
// gather results
(
  // query part for: “water=reservoir”
  node["water"="reservoir"](area.searchArea);
  way["water"="reservoir"](area.searchArea);
  relation["water"="reservoir"](area.searchArea);
  // query part for: “water=lake”
  node["water"="lake"](area.searchArea);
  way["water"="lake"](area.searchArea);
  relation["water"="lake"](area.searchArea);
);
// print results
out body;
>;
out skel qt;
```

Na závěr exportujeme do souboru *voda.geojson*.

In [ ]:
###
df = geopandas.read_file("voda.geojson")
df.head()

In [ ]:
# Konvertujeme dataframe do Krovakova zobrazení (epsg:5514), abychom mohli spočítat plochy
df = df.to_crs("epsg:5514")
df.crs ###

In [ ]:
# vytvoříme sloupec plochy v km2
df["area"] = df.area / 1_000_000
# a vytvoříme také označení, že se jedná o velku plochu, pokud plocha > 1km2
df["large"] = df["area"] > 1
sns.displot(data=df, x="area", col="water", bins=40)

In [ ]:
# Podíváme se na 10 největších vodních děl
df.sort_values("area").tail(10)

In [ ]:
# vykreslíme mapu
fig, ax = plt.subplots(1, 1, figsize=(20, 15)) ###
# velkých
df[df["large"]].plot(ax=ax)
#zvýrazníme hranice
df[df["large"]].boundary.plot(ax=ax, color="0.7")
# a malé také zobrazíme zobrazíme. Jsou vidět?
df[~df["large"]].centroid.plot(ax=ax)

# přidáme basemapu (nezapomenout na crs a zdroj ctx.providers.Stamen.TonerLite)
ctx.add_basemap(ax, crs=df.crs.to_string(), source=ctx.providers.Stamen.TonerLite)

In [ ]:
# Popisky jsou rozmazané. Proto vytvoříme dataframe df2, které má CRS WGS84 (epsg:3857)
df2 = df.to_crs("epsg:3857")
fig, ax = plt.subplots(1, 1, figsize=(20, 10))
df2[df["large"]].plot(ax=ax)
df2[df["large"]].boundary.plot(ax=ax, color="k")
df2[~df["large"]].centroid.plot(ax=ax)

ctx.add_basemap(ax, crs=df2.crs.to_string(), source=ctx.providers.Stamen.TonerLite, zoom=10, alpha=0.9)